# ML MODEL WITH DATA IMPORTED FROM DATABASE

In [1]:
# Importing of dependencies required for analysis
import pandas as pd
import sklearn as skl
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
import requests
import urllib.parse
import datetime
import time
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Read in table of data from PGADMIN or connected database

from sqlalchemy import create_engine
#import library psycopyg2
import psycopg2
#import library sqlio
import pandas.io.sql as sqlio

# Postgres username, password, and database name
POSTGRES_ADDRESS = 'db1-realestate-prediction.cc3utfionj0h.us-east-2.rds.amazonaws.com' ## INSERT YOUR DB ADDRESS
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'realestate'
POSTGRES_PASSWORD = '12345678'
POSTGRES_DBNAME = 'postgres'

#create database connection variable 
conn = psycopg2.connect(user=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, host=POSTGRES_ADDRESS, database=POSTGRES_DBNAME)

/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
# read database from sql database
df = sqlio.read_sql_query('''SELECT * FROM public."REALESTATE";''', conn)
df

,municipality,community,list_price,sold_price,type,style,br,er,wr,fr,kitchen,gar_type,ac,heat,contract_date,mls_id,address,lat,lng
0,Toronto C00,Niagara,699999.0,780000.0,Att/Row/Townhouse,2-Storey,3.0,0.0,1.0,N,1.0,None,Central Air,Gas,2021-01-13,C5083064,"793 Adelaide St W, Toronto",43.643807,-79.409426
1,Toronto C01,University,799000.0,1225000.0,Semi-Detached,2-Storey,3.0,0.0,2.0,Y,2.0,None,None,Gas,2020-05-29,C4773721,"75 Major St, Toronto",43.659648,-79.403456
2,Toronto C01,University,799000.0,1203000.0,Semi-Detached,2-Storey,3.0,1.0,3.0,N,2.0,None,None,Gas,2020-11-05,C4979474,"81 Major St, Toronto",43.659777,-79.403505
3,Toronto C01,Niagara,899000.0,959000.0,Att/Row/Townhouse,2-Storey,3.0,0.0,2.0,Y,1.0,None,Central Air,Gas,2020-09-22,C4923168,"784 Adelaide St W, Toronto",43.644117,-79.409249
4,Toronto C01,Niagara,899000.0,1073000.0,Att/Row/Townhouse,2-Storey,2.0,0.0,2.0,N,1.0,None,Central Air,Gas,2020-11-10,C4985281,"16 Whitaker Ave, Toronto",43.643948,-79.406457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30713,Toronto W10,Thistletown-Beaumonde Heights,1299900.0,1300000.0,Detached,1 1/2 Storey,4.0,2.0,3.0,N,1.0,Attached,Central Air,Gas,2021-10-03,W5391840,"8 Wardlaw Cres, Toronto",43.739635,-79.565621
30714,Toronto W10,Thistletown-Beaumonde Heights,1349000.0,1365000.0,Detached,Backsplit,4.0,1.0,2.0,N,1.0,Attached,Central Air,Gas,2021-11-12,W5430335,"156 Thistle Down Blvd, Toronto",43.740987,-79.550545
30715,Toronto W10,West Humber-Clairville,1599900.0,1600000.0,Detached,2-Storey,4.0,0.0,4.0,Y,1.0,Attached,Central Air,Gas,2021-12-17,W5458498,"11 Woodlot Cres, Toronto",43.732902,-79.614069
30716,Toronto W10,Thistletown-Beaumonde Heights,1750000.0,1850000.0,Detached,Backsplit,5.0,0.0,4.0,Y,1.0,Attached,Central Air,Gas,2021-12-02,W5447590,"16 Forest Path Crt, Toronto",43.741928,-79.555623


In [4]:
df.isnull().sum()

municipality     0
community        0
list_price       0
sold_price       0
type             0
style            0
br               0
er               0
wr               0
fr               0
kitchen          0
gar_type         0
ac               0
heat             0
contract_date    0
mls_id           0
address          0
lat              0
lng              0
dtype: int64

In [5]:
df.dtypes

municipality      object
community         object
list_price       float64
sold_price       float64
type              object
style             object
br               float64
er               float64
wr               float64
fr                object
kitchen          float64
gar_type          object
ac                object
heat              object
contract_date     object
mls_id            object
address           object
lat              float64
lng              float64
dtype: object

# PreProcessing

In [6]:
# preprocessing including converting contract date from object type to date time and back to numeric
# then it can be scaled with other X features

df['contract_date'] = pd.to_datetime(df['contract_date'])
import datetime
import time
z=[]
for i in df['contract_date']:
    y = time.mktime(i.timetuple())
    z.append(y)
    
# contract date column is set as numeric values array
df['contract_date']=z


# drop mls and address columns as they are not objects and we have lat/lng to represent location
df.drop(['mls_id', 'address'], inplace=True, axis=1)

df

,municipality,community,list_price,sold_price,type,style,br,er,wr,fr,kitchen,gar_type,ac,heat,contract_date,lat,lng
0,Toronto C00,Niagara,699999.0,780000.0,Att/Row/Townhouse,2-Storey,3.0,0.0,1.0,N,1.0,None,Central Air,Gas,1.610514e+09,43.643807,-79.409426
1,Toronto C01,University,799000.0,1225000.0,Semi-Detached,2-Storey,3.0,0.0,2.0,Y,2.0,None,None,Gas,1.590725e+09,43.659648,-79.403456
2,Toronto C01,University,799000.0,1203000.0,Semi-Detached,2-Storey,3.0,1.0,3.0,N,2.0,None,None,Gas,1.604552e+09,43.659777,-79.403505
3,Toronto C01,Niagara,899000.0,959000.0,Att/Row/Townhouse,2-Storey,3.0,0.0,2.0,Y,1.0,None,Central Air,Gas,1.600747e+09,43.644117,-79.409249
4,Toronto C01,Niagara,899000.0,1073000.0,Att/Row/Townhouse,2-Storey,2.0,0.0,2.0,N,1.0,None,Central Air,Gas,1.604984e+09,43.643948,-79.406457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30713,Toronto W10,Thistletown-Beaumonde Heights,1299900.0,1300000.0,Detached,1 1/2 Storey,4.0,2.0,3.0,N,1.0,Attached,Central Air,Gas,1.633234e+09,43.739635,-79.565621
30714,Toronto W10,Thistletown-Beaumonde Heights,1349000.0,1365000.0,Detached,Backsplit,4.0,1.0,2.0,N,1.0,Attached,Central Air,Gas,1.636693e+09,43.740987,-79.550545
30715,Toronto W10,West Humber-Clairville,1599900.0,1600000.0,Detached,2-Storey,4.0,0.0,4.0,Y,1.0,Attached,Central Air,Gas,1.639717e+09,43.732902,-79.614069
30716,Toronto W10,Thistletown-Beaumonde Heights,1750000.0,1850000.0,Detached,Backsplit,5.0,0.0,4.0,Y,1.0,Attached,Central Air,Gas,1.638421e+09,43.741928,-79.555623


In [7]:
# array of categorical variables or object data type columns for categorical coding
cat = df.dtypes[df.dtypes == "object"].index.tolist()
cat

['municipality', 'community', 'type', 'style', 'fr', 'gar_type', 'ac', 'heat']

In [8]:
# define ordinal encoding
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
# transform data
# Fit and transform the Ordinal encoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = df[cat].columns

# merge encoded df into original df
final_df = df.merge(encode_df,left_index=True, right_index=True)
final_df

,municipality_x,community_x,list_price,sold_price,type_x,style_x,br,er,wr,fr_x,...,lat,lng,municipality_y,community_y,type_y,style_y,fr_y,gar_type_y,ac_y,heat_y
0,Toronto C00,Niagara,699999.0,780000.0,Att/Row/Townhouse,2-Storey,3.0,0.0,1.0,N,...,43.643807,-79.409426,0.0,91.0,0.0,2.0,0.0,4.0,0.0,1.0
1,Toronto C01,University,799000.0,1225000.0,Semi-Detached,2-Storey,3.0,0.0,2.0,Y,...,43.659648,-79.403456,1.0,121.0,8.0,2.0,1.0,4.0,1.0,1.0
2,Toronto C01,University,799000.0,1203000.0,Semi-Detached,2-Storey,3.0,1.0,3.0,N,...,43.659777,-79.403505,1.0,121.0,8.0,2.0,0.0,4.0,1.0,1.0
3,Toronto C01,Niagara,899000.0,959000.0,Att/Row/Townhouse,2-Storey,3.0,0.0,2.0,Y,...,43.644117,-79.409249,1.0,91.0,0.0,2.0,1.0,4.0,0.0,1.0
4,Toronto C01,Niagara,899000.0,1073000.0,Att/Row/Townhouse,2-Storey,2.0,0.0,2.0,N,...,43.643948,-79.406457,1.0,91.0,0.0,2.0,0.0,4.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30713,Toronto W10,Thistletown-Beaumonde Heights,1299900.0,1300000.0,Detached,1 1/2 Storey,4.0,2.0,3.0,N,...,43.739635,-79.565621,35.0,119.0,1.0,0.0,0.0,0.0,0.0,1.0
30714,Toronto W10,Thistletown-Beaumonde Heights,1349000.0,1365000.0,Detached,Backsplit,4.0,1.0,2.0,N,...,43.740987,-79.550545,35.0,119.0,1.0,4.0,0.0,0.0,0.0,1.0
30715,Toronto W10,West Humber-Clairville,1599900.0,1600000.0,Detached,2-Storey,4.0,0.0,4.0,Y,...,43.732902,-79.614069,35.0,126.0,1.0,2.0,1.0,0.0,0.0,1.0
30716,Toronto W10,Thistletown-Beaumonde Heights,1750000.0,1850000.0,Detached,Backsplit,5.0,0.0,4.0,Y,...,43.741928,-79.555623,35.0,119.0,1.0,4.0,1.0,0.0,0.0,1.0


In [9]:
final_df.columns

Index(['municipality_x', 'community_x', 'list_price', 'sold_price', 'type_x',
       'style_x', 'br', 'er', 'wr', 'fr_x', 'kitchen', 'gar_type_x', 'ac_x',
       'heat_x', 'contract_date', 'lat', 'lng', 'municipality_y',
       'community_y', 'type_y', 'style_y', 'fr_y', 'gar_type_y', 'ac_y',
       'heat_y'],
      dtype='object')

In [10]:
# drop non categorically encoded columns

final_df = final_df.drop(['municipality_x', 'community_x', 'type_x', 'style_x', 'fr_x', 'gar_type_x', 'ac_x', 'heat_x'] ,1)

/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
final_df

,list_price,sold_price,br,er,wr,kitchen,contract_date,lat,lng,municipality_y,community_y,type_y,style_y,fr_y,gar_type_y,ac_y,heat_y
0,699999.0,780000.0,3.0,0.0,1.0,1.0,1.610514e+09,43.643807,-79.409426,0.0,91.0,0.0,2.0,0.0,4.0,0.0,1.0
1,799000.0,1225000.0,3.0,0.0,2.0,2.0,1.590725e+09,43.659648,-79.403456,1.0,121.0,8.0,2.0,1.0,4.0,1.0,1.0
2,799000.0,1203000.0,3.0,1.0,3.0,2.0,1.604552e+09,43.659777,-79.403505,1.0,121.0,8.0,2.0,0.0,4.0,1.0,1.0
3,899000.0,959000.0,3.0,0.0,2.0,1.0,1.600747e+09,43.644117,-79.409249,1.0,91.0,0.0,2.0,1.0,4.0,0.0,1.0
4,899000.0,1073000.0,2.0,0.0,2.0,1.0,1.604984e+09,43.643948,-79.406457,1.0,91.0,0.0,2.0,0.0,4.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30713,1299900.0,1300000.0,4.0,2.0,3.0,1.0,1.633234e+09,43.739635,-79.565621,35.0,119.0,1.0,0.0,0.0,0.0,0.0,1.0
30714,1349000.0,1365000.0,4.0,1.0,2.0,1.0,1.636693e+09,43.740987,-79.550545,35.0,119.0,1.0,4.0,0.0,0.0,0.0,1.0
30715,1599900.0,1600000.0,4.0,0.0,4.0,1.0,1.639717e+09,43.732902,-79.614069,35.0,126.0,1.0,2.0,1.0,0.0,0.0,1.0
30716,1750000.0,1850000.0,5.0,0.0,4.0,1.0,1.638421e+09,43.741928,-79.555623,35.0,119.0,1.0,4.0,1.0,0.0,0.0,1.0


In [12]:
# Split our preprocessed data into our features and target arrays
y = final_df["sold_price"].values
X = final_df.drop(["sold_price"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Linear Regression

In [14]:
#Liner Regression model created and trained on training data and tested on test data
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()  
regressor.fit(X_train_scaled, y_train)

# predict values on test data
y_predd = regressor.predict(X_test_scaled)

In [15]:
# print all MSE and variance scores

from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np

print('Mean Absolute Error: {:.2f}'.format(metrics.mean_absolute_error(y_test, y_predd))) 
print('Mean Squared Error:{:.2f}'.format(metrics.mean_squared_error(y_test, y_predd)))  
print('Root Mean Squared Error:{:.2f}'.format(np.sqrt(metrics.mean_squared_error(y_test, y_predd))))
print('Variance score is: {:.2f}'.format(metrics.explained_variance_score(y_test,y_predd)))

Mean Absolute Error: 111670.42
Mean Squared Error:24120628569.62
Root Mean Squared Error:155308.17
Variance score is: 0.97


In [16]:
# print accuracy scores and R^2 values
print('Linear Regression Model')
print("Train Score: {:.2f}".format(regressor.score(X_train_scaled,y_train)))
print("Test Score: {:.2f}".format(regressor.score(X_test_scaled, y_test)))
print("R^2 Score: {:.2f}".format(r2_score(y_test,y_predd)))

Linear Regression Model
Train Score: 0.97
Test Score: 0.97
R^2 Score: 0.97


In [17]:
# actual score of regression model not rounded off
regressor.score(X_test_scaled, y_test)

0.9689755784505494

In [18]:
# print Coeef
importance = regressor.coef_
importance

array([ 8.95267829e+05,  8.07938757e+03, -3.53932913e+03,  4.42718986e+03,
       -1.11820143e+04,  3.99145773e+04, -2.73994078e+03, -2.99419181e+03,
       -1.91396611e+04,  2.07762431e+03,  8.99538802e+02, -8.04519831e+03,
        2.09308394e+03,  8.77792404e+03,  3.36833940e+03, -8.79246483e+02])

# Add model predicted values to original dataframe

In [19]:
# create predictions using model on entire dataset
# Create a StandardScaler instance
scaler_i = StandardScaler()

# Fit the StandardScaler
X_scaler_i = scaler_i.fit(X)

# Scale the data
X_total_scaled = X_scaler_i.transform(X)

In [20]:
# predict values on total data
predictions = regressor.predict(X_total_scaled)

In [21]:
# score of accuracy of regressor model running on dataset
print("R^2 Score: {:.2f}".format(r2_score(y,predictions)))

R^2 Score: 0.97


In [22]:
len(predictions)

30718

In [23]:
# add predictions to df as prediction column
# read database from sql database
x = sqlio.read_sql_query('''SELECT * FROM public."REALESTATE";''', conn)
x['sold_price_predictions']=predictions

In [24]:
x

,municipality,community,list_price,sold_price,type,style,br,er,wr,fr,kitchen,gar_type,ac,heat,contract_date,mls_id,address,lat,lng,sold_price_predictions
0,Toronto C00,Niagara,699999.0,780000.0,Att/Row/Townhouse,2-Storey,3.0,0.0,1.0,N,1.0,None,Central Air,Gas,2021-01-13,C5083064,"793 Adelaide St W, Toronto",43.643807,-79.409426,8.836353e+05
1,Toronto C01,University,799000.0,1225000.0,Semi-Detached,2-Storey,3.0,0.0,2.0,Y,2.0,None,None,Gas,2020-05-29,C4773721,"75 Major St, Toronto",43.659648,-79.403456,9.174046e+05
2,Toronto C01,University,799000.0,1203000.0,Semi-Detached,2-Storey,3.0,1.0,3.0,N,2.0,None,None,Gas,2020-11-05,C4979474,"81 Major St, Toronto",43.659777,-79.403505,9.453127e+05
3,Toronto C01,Niagara,899000.0,959000.0,Att/Row/Townhouse,2-Storey,3.0,0.0,2.0,Y,1.0,None,Central Air,Gas,2020-09-22,C4923168,"784 Adelaide St W, Toronto",43.644117,-79.409249,1.048372e+06
4,Toronto C01,Niagara,899000.0,1073000.0,Att/Row/Townhouse,2-Storey,2.0,0.0,2.0,N,1.0,None,Central Air,Gas,2020-11-10,C4985281,"16 Whitaker Ave, Toronto",43.643948,-79.406457,1.045383e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30713,Toronto W10,Thistletown-Beaumonde Heights,1299900.0,1300000.0,Detached,1 1/2 Storey,4.0,2.0,3.0,N,1.0,Attached,Central Air,Gas,2021-10-03,W5391840,"8 Wardlaw Cres, Toronto",43.739635,-79.565621,1.413590e+06
30714,Toronto W10,Thistletown-Beaumonde Heights,1349000.0,1365000.0,Detached,Backsplit,4.0,1.0,2.0,N,1.0,Attached,Central Air,Gas,2021-11-12,W5430335,"156 Thistle Down Blvd, Toronto",43.740987,-79.550545,1.455300e+06
30715,Toronto W10,West Humber-Clairville,1599900.0,1600000.0,Detached,2-Storey,4.0,0.0,4.0,Y,1.0,Attached,Central Air,Gas,2021-12-17,W5458498,"11 Woodlot Cres, Toronto",43.732902,-79.614069,1.714175e+06
30716,Toronto W10,Thistletown-Beaumonde Heights,1750000.0,1850000.0,Detached,Backsplit,5.0,0.0,4.0,Y,1.0,Attached,Central Air,Gas,2021-12-02,W5447590,"16 Forest Path Crt, Toronto",43.741928,-79.555623,1.850858e+06


In [25]:
x.dtypes

municipality               object
community                  object
list_price                float64
sold_price                float64
type                       object
style                      object
br                        float64
er                        float64
wr                        float64
fr                         object
kitchen                   float64
gar_type                   object
ac                         object
heat                       object
contract_date              object
mls_id                     object
address                    object
lat                       float64
lng                       float64
sold_price_predictions    float64
dtype: object

In [26]:
# convert scientific notation to numeric
x['sold_price_predictions']=x['sold_price_predictions'].apply(lambda x: '%.5f' % x)

In [27]:
x

,municipality,community,list_price,sold_price,type,style,br,er,wr,fr,kitchen,gar_type,ac,heat,contract_date,mls_id,address,lat,lng,sold_price_predictions
0,Toronto C00,Niagara,699999.0,780000.0,Att/Row/Townhouse,2-Storey,3.0,0.0,1.0,N,1.0,None,Central Air,Gas,2021-01-13,C5083064,"793 Adelaide St W, Toronto",43.643807,-79.409426,883635.34576
1,Toronto C01,University,799000.0,1225000.0,Semi-Detached,2-Storey,3.0,0.0,2.0,Y,2.0,None,None,Gas,2020-05-29,C4773721,"75 Major St, Toronto",43.659648,-79.403456,917404.58667
2,Toronto C01,University,799000.0,1203000.0,Semi-Detached,2-Storey,3.0,1.0,3.0,N,2.0,None,None,Gas,2020-11-05,C4979474,"81 Major St, Toronto",43.659777,-79.403505,945312.72174
3,Toronto C01,Niagara,899000.0,959000.0,Att/Row/Townhouse,2-Storey,3.0,0.0,2.0,Y,1.0,None,Central Air,Gas,2020-09-22,C4923168,"784 Adelaide St W, Toronto",43.644117,-79.409249,1048371.82796
4,Toronto C01,Niagara,899000.0,1073000.0,Att/Row/Townhouse,2-Storey,2.0,0.0,2.0,N,1.0,None,Central Air,Gas,2020-11-10,C4985281,"16 Whitaker Ave, Toronto",43.643948,-79.406457,1045383.45692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30713,Toronto W10,Thistletown-Beaumonde Heights,1299900.0,1300000.0,Detached,1 1/2 Storey,4.0,2.0,3.0,N,1.0,Attached,Central Air,Gas,2021-10-03,W5391840,"8 Wardlaw Cres, Toronto",43.739635,-79.565621,1413590.48615
30714,Toronto W10,Thistletown-Beaumonde Heights,1349000.0,1365000.0,Detached,Backsplit,4.0,1.0,2.0,N,1.0,Attached,Central Air,Gas,2021-11-12,W5430335,"156 Thistle Down Blvd, Toronto",43.740987,-79.550545,1455299.79045
30715,Toronto W10,West Humber-Clairville,1599900.0,1600000.0,Detached,2-Storey,4.0,0.0,4.0,Y,1.0,Attached,Central Air,Gas,2021-12-17,W5458498,"11 Woodlot Cres, Toronto",43.732902,-79.614069,1714174.52640
30716,Toronto W10,Thistletown-Beaumonde Heights,1750000.0,1850000.0,Detached,Backsplit,5.0,0.0,4.0,Y,1.0,Attached,Central Air,Gas,2021-12-02,W5447590,"16 Forest Path Crt, Toronto",43.741928,-79.555623,1850857.69048


In [28]:
# output csv to resources cleaned column
output_data_file_new = "Resources_Cleaned/Inactive_Predictions.csv"
x.to_csv(output_data_file_new, index=False)

# Model predicts on testing data for active listings

In [29]:
df2 = pd.read_csv('Resources_Uncleaned/Active_Listings_Raw.csv')
df2

,LSC,EC,St#,Street Name,Abbr,Dir,Municipality,Community,List Price,Type,...,Fam,Kit,Gar Type,(A/C),Heat,Contract Date,List Brokerage,Co-Op Brokerage,CB Com,MLS #
0,New,NaN,8 Virgilwood Dr,NaN,NaN,NaN,Toronto C07,Westminster-Branson,"$989,000",Semi-Detached,...,N,2.0,None,Central Air,Gas,12/17/2021,RE/MAX PREMIER I...,416-987-8000,2.25%,C5457674
1,New,NaN,213 Northcliffe Blvd,NaN,NaN,NaN,Toronto C03,Oakwood-Vaughan,"$999,900",Detached,...,N,1.0,Detached,Window Unit,Gas,1/9/2022,RE/MAX REALTRON ...,905-470-9800,2.50%,C5467990
2,New,NaN,144 Winnett Ave,NaN,NaN,NaN,Toronto C03,Humewood-Cedarvale,"$1,038,000",Semi-Detached,...,N,1.0,None,Central Air,Gas,1/10/2022,ROYAL LEPAGE/J &...,416-489-2121,2.5% + H.S...,C5468205
3,New,NaN,357 Oakwood Ave,NaN,NaN,NaN,Toronto C03,Oakwood-Vaughan,"$1,145,000",Store w/Apt/Offc,...,N,3.0,None,Wall Unit,Gas,10/13/2021,FOREST HILL REAL...,905-237-7427,2.5%+Hst,C5400499
4,New,NaN,82 Petman Ave,NaN,NaN,NaN,Toronto C10,Mount Pleasant East,"$1,149,000",Detached,...,N,1.0,None,Central Air,Gas,1/11/2022,CHESTNUT PARK RE...,416-925-9191,2.5% + Hst,C5469304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557,New,NaN,9 Amaron Ave,NaN,NaN,NaN,Toronto W10,Thistletown-Beaumonde Heights,"$1,049,900",Detached,...,Y,1.0,Detached,Central Air,Gas,1/10/2022,RE/MAX PREMIER I...,416-743-2000,2.5 %,W5468541
558,New,NaN,35 Chilcot Ave,NaN,NaN,NaN,Toronto W10,Elms-Old Rexdale,"$1,199,999",Detached,...,Y,1.0,Detached,Central Air,Gas,11/27/2021,HOMELIFE LANDMAR...,905-305-1600,2.50%,W5443512
559,New,NaN,145 Silverstone Dr,NaN,NaN,NaN,Toronto W10,Mount Olive-Silverstone-Jamestown,"$1,231,000",Detached,...,Y,1.0,Detached,Central Air,Gas,1/7/2022,HOMELIFE/MIRACLE...,416-747-9777,2.5%-199Mf,W5467290
560,New,NaN,107 Mount Olive Dr,NaN,NaN,NaN,Toronto W10,Mount Olive-Silverstone-Jamestown,"$1,279,000",Detached,...,N,1.0,Attached,Central Air,Gas,1/8/2022,RE/MAX COMMUNITY...,416-287-2222,2.50%,W5467611


In [30]:
# do same cleaning process for previous dataframe

In [31]:
# Check for null values within the dataframe
df2.isnull().sum()

LSC                  0
EC                 562
St#                  0
Street Name        562
Abbr               562
Dir                562
Municipality         0
Community            0
List Price           0
Type                 0
Style               10
BR                  10
(+)                214
Wr                   0
Fam                 14
Kit                 10
Gar Type             7
(A/C)                9
Heat                 9
Contract Date        0
List Brokerage       0
Co-Op Brokerage      0
CB Com               0
MLS #                0
dtype: int64

In [32]:
# Convert all NaNs in the Extra Room column / (+) to 0s
df2['(+)'] = df2['(+)'].fillna(0)

# drop LSC, EC, Street Names, Abbr, Dir, List/Coop Brokerage and Sold Date.
# We remove sold date as feature as sold date does not exist in active listings dataset(the data we want predictions on)
df2.drop(['LSC', 'EC', 'Street Name', 'Abbr', 'Dir', 'List Brokerage', 'Co-Op Brokerage', 'CB Com'], axis=1, inplace=True)

In [33]:
df2.isnull().sum()

St#                0
Municipality       0
Community          0
List Price         0
Type               0
Style             10
BR                10
(+)                0
Wr                 0
Fam               14
Kit               10
Gar Type           7
(A/C)              9
Heat               9
Contract Date      0
MLS #              0
dtype: int64

In [34]:
# drop all null values
df2 = df2.dropna()
df2.isnull().sum()

St#               0
Municipality      0
Community         0
List Price        0
Type              0
Style             0
BR                0
(+)               0
Wr                0
Fam               0
Kit               0
Gar Type          0
(A/C)             0
Heat              0
Contract Date     0
MLS #             0
dtype: int64

In [35]:
df2

,St#,Municipality,Community,List Price,Type,Style,BR,(+),Wr,Fam,Kit,Gar Type,(A/C),Heat,Contract Date,MLS #
0,8 Virgilwood Dr,Toronto C07,Westminster-Branson,"$989,000",Semi-Detached,Bungalow,4.0,2.0,2,N,2.0,None,Central Air,Gas,12/17/2021,C5457674
1,213 Northcliffe Blvd,Toronto C03,Oakwood-Vaughan,"$999,900",Detached,2-Storey,3.0,0.0,2,N,1.0,Detached,Window Unit,Gas,1/9/2022,C5467990
2,144 Winnett Ave,Toronto C03,Humewood-Cedarvale,"$1,038,000",Semi-Detached,2-Storey,3.0,0.0,3,N,1.0,None,Central Air,Gas,1/10/2022,C5468205
3,357 Oakwood Ave,Toronto C03,Oakwood-Vaughan,"$1,145,000",Store w/Apt/Offc,2-Storey,3.0,0.0,3,N,3.0,None,Wall Unit,Gas,10/13/2021,C5400499
4,82 Petman Ave,Toronto C10,Mount Pleasant East,"$1,149,000",Detached,2-Storey,3.0,0.0,2,N,1.0,None,Central Air,Gas,1/11/2022,C5469304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557,9 Amaron Ave,Toronto W10,Thistletown-Beaumonde Heights,"$1,049,900",Detached,Bungalow,3.0,1.0,3,Y,1.0,Detached,Central Air,Gas,1/10/2022,W5468541
558,35 Chilcot Ave,Toronto W10,Elms-Old Rexdale,"$1,199,999",Detached,1 1/2 Storey,3.0,2.0,2,Y,1.0,Detached,Central Air,Gas,11/27/2021,W5443512
559,145 Silverstone Dr,Toronto W10,Mount Olive-Silverstone-Jamestown,"$1,231,000",Detached,Bungalow,4.0,2.0,2,Y,1.0,Detached,Central Air,Gas,1/7/2022,W5467290
560,107 Mount Olive Dr,Toronto W10,Mount Olive-Silverstone-Jamestown,"$1,279,000",Detached,2-Storey,4.0,1.0,3,N,1.0,Attached,Central Air,Gas,1/8/2022,W5467611


In [36]:
# rename some columns
df2 = df2.rename(columns={'St#': 'Address', '(+)':'ER', 'Wr': 'WR', 'Fam': 'FR', 'Kit':'Kitchen', 'Contract Date ': 'Contract_Date', 'Gar Type': 'Gar_Type', '(A/C)':'AC'})

In [37]:
# Add Toronto to all the street addresses so when the code to find lat/lng runs it will only find addresses in GTA
df2['Address'] = df2['Address'].astype(str) + ', Toronto'

In [38]:
# resetting the DataFrame index
df2 = df2.reset_index()
df2.drop(['index'], axis=1, inplace=True)

# convert time to date time
df2['Contract_Date'] = pd.to_datetime(df2['Contract_Date'])
df2 = df2.rename(columns={'List Price': 'List_Price($)'})
df2.dtypes

Address                  object
Municipality             object
Community                object
List_Price($)            object
Type                     object
Style                    object
BR                      float64
ER                      float64
WR                        int64
FR                       object
Kitchen                 float64
Gar_Type                 object
AC                       object
Heat                     object
Contract_Date    datetime64[ns]
MLS #                    object
dtype: object

In [39]:
# find lat/lng for these 547 rows and add them to a dataframe
data=[]
s=[]
f=[]
for i in df2['Address']:
    try: 
        url = "https://nominatim.openstreetmap.org/?addressdetails=1&q=" + i + "+" +"&format=json&limit=1"
        response = requests.get(url).json()
        data.append({"Address":i, "Lat": response[0]["lat"], "Lng": response[0]["lon"]})
        s.append(i)
    except:
        data.append({"Address":i, "Lat": np.NaN, "Lng": np.NaN})
        f.append(i)

In [40]:
print(len(s))
print(len(f))

530
17


In [41]:
# use lat/lng data and convert to dataframe and check for null values
data2_df = pd.DataFrame(data)
data2_df.isnull().sum()

Address     0
Lat        17
Lng        17
dtype: int64

In [42]:
# print dataframe
data2_df

,Address,Lat,Lng
0,"8 Virgilwood Dr, Toronto",43.773367,-79.4485395
1,"213 Northcliffe Blvd, Toronto",43.682526175,-79.4422434
2,"144 Winnett Ave, Toronto",43.68782725,-79.43277225
3,"357 Oakwood Ave, Toronto",43.68699204,-79.43844438
4,"82 Petman Ave, Toronto",43.70916151428571,-79.38486365714286
...,...,...,...
542,"9 Amaron Ave, Toronto",43.7418921625,-79.5787170875
543,"35 Chilcot Ave, Toronto",43.7141594,-79.5488803
544,"145 Silverstone Dr, Toronto",43.751753671986975,-79.59027921260754
545,"107 Mount Olive Dr, Toronto",43.7462574,-79.5919673


In [43]:
# print dataframe
df2

,Address,Municipality,Community,List_Price($),Type,Style,BR,ER,WR,FR,Kitchen,Gar_Type,AC,Heat,Contract_Date,MLS #
0,"8 Virgilwood Dr, Toronto",Toronto C07,Westminster-Branson,"$989,000",Semi-Detached,Bungalow,4.0,2.0,2,N,2.0,None,Central Air,Gas,2021-12-17,C5457674
1,"213 Northcliffe Blvd, Toronto",Toronto C03,Oakwood-Vaughan,"$999,900",Detached,2-Storey,3.0,0.0,2,N,1.0,Detached,Window Unit,Gas,2022-01-09,C5467990
2,"144 Winnett Ave, Toronto",Toronto C03,Humewood-Cedarvale,"$1,038,000",Semi-Detached,2-Storey,3.0,0.0,3,N,1.0,None,Central Air,Gas,2022-01-10,C5468205
3,"357 Oakwood Ave, Toronto",Toronto C03,Oakwood-Vaughan,"$1,145,000",Store w/Apt/Offc,2-Storey,3.0,0.0,3,N,3.0,None,Wall Unit,Gas,2021-10-13,C5400499
4,"82 Petman Ave, Toronto",Toronto C10,Mount Pleasant East,"$1,149,000",Detached,2-Storey,3.0,0.0,2,N,1.0,None,Central Air,Gas,2022-01-11,C5469304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,"9 Amaron Ave, Toronto",Toronto W10,Thistletown-Beaumonde Heights,"$1,049,900",Detached,Bungalow,3.0,1.0,3,Y,1.0,Detached,Central Air,Gas,2022-01-10,W5468541
543,"35 Chilcot Ave, Toronto",Toronto W10,Elms-Old Rexdale,"$1,199,999",Detached,1 1/2 Storey,3.0,2.0,2,Y,1.0,Detached,Central Air,Gas,2021-11-27,W5443512
544,"145 Silverstone Dr, Toronto",Toronto W10,Mount Olive-Silverstone-Jamestown,"$1,231,000",Detached,Bungalow,4.0,2.0,2,Y,1.0,Detached,Central Air,Gas,2022-01-07,W5467290
545,"107 Mount Olive Dr, Toronto",Toronto W10,Mount Olive-Silverstone-Jamestown,"$1,279,000",Detached,2-Storey,4.0,1.0,3,N,1.0,Attached,Central Air,Gas,2022-01-08,W5467611


In [44]:
# merge these two dataframes into each other. 
merge2_df = df2.merge(data2_df, how = 'inner', left_index=True, right_index=True)
merge2_df

,Address_x,Municipality,Community,List_Price($),Type,Style,BR,ER,WR,FR,Kitchen,Gar_Type,AC,Heat,Contract_Date,MLS #,Address_y,Lat,Lng
0,"8 Virgilwood Dr, Toronto",Toronto C07,Westminster-Branson,"$989,000",Semi-Detached,Bungalow,4.0,2.0,2,N,2.0,None,Central Air,Gas,2021-12-17,C5457674,"8 Virgilwood Dr, Toronto",43.773367,-79.4485395
1,"213 Northcliffe Blvd, Toronto",Toronto C03,Oakwood-Vaughan,"$999,900",Detached,2-Storey,3.0,0.0,2,N,1.0,Detached,Window Unit,Gas,2022-01-09,C5467990,"213 Northcliffe Blvd, Toronto",43.682526175,-79.4422434
2,"144 Winnett Ave, Toronto",Toronto C03,Humewood-Cedarvale,"$1,038,000",Semi-Detached,2-Storey,3.0,0.0,3,N,1.0,None,Central Air,Gas,2022-01-10,C5468205,"144 Winnett Ave, Toronto",43.68782725,-79.43277225
3,"357 Oakwood Ave, Toronto",Toronto C03,Oakwood-Vaughan,"$1,145,000",Store w/Apt/Offc,2-Storey,3.0,0.0,3,N,3.0,None,Wall Unit,Gas,2021-10-13,C5400499,"357 Oakwood Ave, Toronto",43.68699204,-79.43844438
4,"82 Petman Ave, Toronto",Toronto C10,Mount Pleasant East,"$1,149,000",Detached,2-Storey,3.0,0.0,2,N,1.0,None,Central Air,Gas,2022-01-11,C5469304,"82 Petman Ave, Toronto",43.70916151428571,-79.38486365714286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,"9 Amaron Ave, Toronto",Toronto W10,Thistletown-Beaumonde Heights,"$1,049,900",Detached,Bungalow,3.0,1.0,3,Y,1.0,Detached,Central Air,Gas,2022-01-10,W5468541,"9 Amaron Ave, Toronto",43.7418921625,-79.5787170875
543,"35 Chilcot Ave, Toronto",Toronto W10,Elms-Old Rexdale,"$1,199,999",Detached,1 1/2 Storey,3.0,2.0,2,Y,1.0,Detached,Central Air,Gas,2021-11-27,W5443512,"35 Chilcot Ave, Toronto",43.7141594,-79.5488803
544,"145 Silverstone Dr, Toronto",Toronto W10,Mount Olive-Silverstone-Jamestown,"$1,231,000",Detached,Bungalow,4.0,2.0,2,Y,1.0,Detached,Central Air,Gas,2022-01-07,W5467290,"145 Silverstone Dr, Toronto",43.751753671986975,-79.59027921260754
545,"107 Mount Olive Dr, Toronto",Toronto W10,Mount Olive-Silverstone-Jamestown,"$1,279,000",Detached,2-Storey,4.0,1.0,3,N,1.0,Attached,Central Air,Gas,2022-01-08,W5467611,"107 Mount Olive Dr, Toronto",43.7462574,-79.5919673


In [45]:
merge2_df.dtypes

Address_x                object
Municipality             object
Community                object
List_Price($)            object
Type                     object
Style                    object
BR                      float64
ER                      float64
WR                        int64
FR                       object
Kitchen                 float64
Gar_Type                 object
AC                       object
Heat                     object
Contract_Date    datetime64[ns]
MLS #                    object
Address_y                object
Lat                      object
Lng                      object
dtype: object

In [46]:
# some pre processing done and change column names to same name as dataset read in from database
merge2_df.drop(['Address_x'], axis=1, inplace=True)
merge2_df= merge2_df.rename(columns={'Address_y': 'address', 'Municipality':'municipality','Community':'community',
                                     'List_Price($)': 'list_price', 'Type':'type', 'Style':'style', 'BR':'br','ER':'er','WR':'wr',
                                     'FR':'fr', 'Kitchen':'kitchen','Gar_Type': 'gar_type','AC':'ac','Heat':'heat',
                                    'Contract_Date': 'contract_date', 'Lat':'lat', 'Lng':'lng','MLS #':'mls_id'})
merge2_df

,municipality,community,list_price,type,style,br,er,wr,fr,kitchen,gar_type,ac,heat,contract_date,mls_id,address,lat,lng
0,Toronto C07,Westminster-Branson,"$989,000",Semi-Detached,Bungalow,4.0,2.0,2,N,2.0,None,Central Air,Gas,2021-12-17,C5457674,"8 Virgilwood Dr, Toronto",43.773367,-79.4485395
1,Toronto C03,Oakwood-Vaughan,"$999,900",Detached,2-Storey,3.0,0.0,2,N,1.0,Detached,Window Unit,Gas,2022-01-09,C5467990,"213 Northcliffe Blvd, Toronto",43.682526175,-79.4422434
2,Toronto C03,Humewood-Cedarvale,"$1,038,000",Semi-Detached,2-Storey,3.0,0.0,3,N,1.0,None,Central Air,Gas,2022-01-10,C5468205,"144 Winnett Ave, Toronto",43.68782725,-79.43277225
3,Toronto C03,Oakwood-Vaughan,"$1,145,000",Store w/Apt/Offc,2-Storey,3.0,0.0,3,N,3.0,None,Wall Unit,Gas,2021-10-13,C5400499,"357 Oakwood Ave, Toronto",43.68699204,-79.43844438
4,Toronto C10,Mount Pleasant East,"$1,149,000",Detached,2-Storey,3.0,0.0,2,N,1.0,None,Central Air,Gas,2022-01-11,C5469304,"82 Petman Ave, Toronto",43.70916151428571,-79.38486365714286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,Toronto W10,Thistletown-Beaumonde Heights,"$1,049,900",Detached,Bungalow,3.0,1.0,3,Y,1.0,Detached,Central Air,Gas,2022-01-10,W5468541,"9 Amaron Ave, Toronto",43.7418921625,-79.5787170875
543,Toronto W10,Elms-Old Rexdale,"$1,199,999",Detached,1 1/2 Storey,3.0,2.0,2,Y,1.0,Detached,Central Air,Gas,2021-11-27,W5443512,"35 Chilcot Ave, Toronto",43.7141594,-79.5488803
544,Toronto W10,Mount Olive-Silverstone-Jamestown,"$1,231,000",Detached,Bungalow,4.0,2.0,2,Y,1.0,Detached,Central Air,Gas,2022-01-07,W5467290,"145 Silverstone Dr, Toronto",43.751753671986975,-79.59027921260754
545,Toronto W10,Mount Olive-Silverstone-Jamestown,"$1,279,000",Detached,2-Storey,4.0,1.0,3,N,1.0,Attached,Central Air,Gas,2022-01-08,W5467611,"107 Mount Olive Dr, Toronto",43.7462574,-79.5919673


In [47]:
merge2_df.dtypes

municipality             object
community                object
list_price               object
type                     object
style                    object
br                      float64
er                      float64
wr                        int64
fr                       object
kitchen                 float64
gar_type                 object
ac                       object
heat                     object
contract_date    datetime64[ns]
mls_id                   object
address                  object
lat                      object
lng                      object
dtype: object

In [48]:
# convert list prices to numerical format
merge2_df['list_price'] = merge2_df['list_price'].replace('[\$,]', '', regex=True).astype(float)

# convert lat and lng to numeric
merge2_df["lng"] = pd.to_numeric(merge2_df["lng"])
merge2_df["lat"] = pd.to_numeric(merge2_df["lat"])

In [49]:
merge2_df.isnull().sum()

municipality      0
community         0
list_price        0
type              0
style             0
br                0
er                0
wr                0
fr                0
kitchen           0
gar_type          0
ac                0
heat              0
contract_date     0
mls_id            0
address           0
lat              17
lng              17
dtype: int64

In [50]:
# drop null values and reset index
merge2_df = merge2_df.dropna()

In [51]:
# resetting the DataFrame index
merge2_df = merge2_df.reset_index()

In [52]:
# drop last column which we do not need in our machine learning model
merge2_df.drop(['index'], inplace=True, axis=1)
merge2_df

,municipality,community,list_price,type,style,br,er,wr,fr,kitchen,gar_type,ac,heat,contract_date,mls_id,address,lat,lng
0,Toronto C07,Westminster-Branson,989000.0,Semi-Detached,Bungalow,4.0,2.0,2,N,2.0,None,Central Air,Gas,2021-12-17,C5457674,"8 Virgilwood Dr, Toronto",43.773367,-79.448539
1,Toronto C03,Oakwood-Vaughan,999900.0,Detached,2-Storey,3.0,0.0,2,N,1.0,Detached,Window Unit,Gas,2022-01-09,C5467990,"213 Northcliffe Blvd, Toronto",43.682526,-79.442243
2,Toronto C03,Humewood-Cedarvale,1038000.0,Semi-Detached,2-Storey,3.0,0.0,3,N,1.0,None,Central Air,Gas,2022-01-10,C5468205,"144 Winnett Ave, Toronto",43.687827,-79.432772
3,Toronto C03,Oakwood-Vaughan,1145000.0,Store w/Apt/Offc,2-Storey,3.0,0.0,3,N,3.0,None,Wall Unit,Gas,2021-10-13,C5400499,"357 Oakwood Ave, Toronto",43.686992,-79.438444
4,Toronto C10,Mount Pleasant East,1149000.0,Detached,2-Storey,3.0,0.0,2,N,1.0,None,Central Air,Gas,2022-01-11,C5469304,"82 Petman Ave, Toronto",43.709162,-79.384864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,Toronto W10,Thistletown-Beaumonde Heights,1049900.0,Detached,Bungalow,3.0,1.0,3,Y,1.0,Detached,Central Air,Gas,2022-01-10,W5468541,"9 Amaron Ave, Toronto",43.741892,-79.578717
526,Toronto W10,Elms-Old Rexdale,1199999.0,Detached,1 1/2 Storey,3.0,2.0,2,Y,1.0,Detached,Central Air,Gas,2021-11-27,W5443512,"35 Chilcot Ave, Toronto",43.714159,-79.548880
527,Toronto W10,Mount Olive-Silverstone-Jamestown,1231000.0,Detached,Bungalow,4.0,2.0,2,Y,1.0,Detached,Central Air,Gas,2022-01-07,W5467290,"145 Silverstone Dr, Toronto",43.751754,-79.590279
528,Toronto W10,Mount Olive-Silverstone-Jamestown,1279000.0,Detached,2-Storey,4.0,1.0,3,N,1.0,Attached,Central Air,Gas,2022-01-08,W5467611,"107 Mount Olive Dr, Toronto",43.746257,-79.591967


In [53]:
# now as this dataframe is complete, we can start coding for scaling/encoding

In [54]:
# drop these columns as they will not be features. for address we have lat/lng now
merge2 = merge2_df.drop(['address', 'mls_id'],axis=1)

In [55]:
# convert contract date to numerical object for scaling
a=[]
for i in merge2['contract_date']:
    b = time.mktime(i.timetuple())
    a.append(b)
    
merge2['contract_date']=a
merge2

,municipality,community,list_price,type,style,br,er,wr,fr,kitchen,gar_type,ac,heat,contract_date,lat,lng
0,Toronto C07,Westminster-Branson,989000.0,Semi-Detached,Bungalow,4.0,2.0,2,N,2.0,None,Central Air,Gas,1.639717e+09,43.773367,-79.448539
1,Toronto C03,Oakwood-Vaughan,999900.0,Detached,2-Storey,3.0,0.0,2,N,1.0,Detached,Window Unit,Gas,1.641704e+09,43.682526,-79.442243
2,Toronto C03,Humewood-Cedarvale,1038000.0,Semi-Detached,2-Storey,3.0,0.0,3,N,1.0,None,Central Air,Gas,1.641791e+09,43.687827,-79.432772
3,Toronto C03,Oakwood-Vaughan,1145000.0,Store w/Apt/Offc,2-Storey,3.0,0.0,3,N,3.0,None,Wall Unit,Gas,1.634098e+09,43.686992,-79.438444
4,Toronto C10,Mount Pleasant East,1149000.0,Detached,2-Storey,3.0,0.0,2,N,1.0,None,Central Air,Gas,1.641877e+09,43.709162,-79.384864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,Toronto W10,Thistletown-Beaumonde Heights,1049900.0,Detached,Bungalow,3.0,1.0,3,Y,1.0,Detached,Central Air,Gas,1.641791e+09,43.741892,-79.578717
526,Toronto W10,Elms-Old Rexdale,1199999.0,Detached,1 1/2 Storey,3.0,2.0,2,Y,1.0,Detached,Central Air,Gas,1.637989e+09,43.714159,-79.548880
527,Toronto W10,Mount Olive-Silverstone-Jamestown,1231000.0,Detached,Bungalow,4.0,2.0,2,Y,1.0,Detached,Central Air,Gas,1.641532e+09,43.751754,-79.590279
528,Toronto W10,Mount Olive-Silverstone-Jamestown,1279000.0,Detached,2-Storey,4.0,1.0,3,N,1.0,Attached,Central Air,Gas,1.641618e+09,43.746257,-79.591967


In [56]:
# get categorical variables to encode
cat2 = merge2.dtypes[merge2_df.dtypes == "object"].index.tolist()
cat2

['municipality', 'community', 'type', 'style', 'fr', 'gar_type', 'ac', 'heat']

In [57]:
# encode categorical variables and add to a dataframe
encode2_df = pd.DataFrame(enc.fit_transform(merge2[cat2]))

# Add the encoded variable names to the DataFrame with respective columnn names
encode2_df.columns = merge2[cat2].columns
encode2_df.head()

,municipality,community,type,style,fr,gar_type,ac,heat
0,5.0,111.0,6.0,8.0,0.0,4.0,0.0,1.0
1,2.0,86.0,1.0,2.0,0.0,3.0,4.0,1.0
2,2.0,53.0,6.0,2.0,0.0,4.0,0.0,1.0
3,2.0,86.0,7.0,2.0,0.0,4.0,3.0,1.0
4,8.0,78.0,1.0,2.0,0.0,4.0,0.0,1.0


In [58]:
# merge encoded dataframe to original df
final2_df = merge2.merge(encode2_df,left_index=True, right_index=True)
final2_df

,municipality_x,community_x,list_price,type_x,style_x,br,er,wr,fr_x,kitchen,...,lat,lng,municipality_y,community_y,type_y,style_y,fr_y,gar_type_y,ac_y,heat_y
0,Toronto C07,Westminster-Branson,989000.0,Semi-Detached,Bungalow,4.0,2.0,2,N,2.0,...,43.773367,-79.448539,5.0,111.0,6.0,8.0,0.0,4.0,0.0,1.0
1,Toronto C03,Oakwood-Vaughan,999900.0,Detached,2-Storey,3.0,0.0,2,N,1.0,...,43.682526,-79.442243,2.0,86.0,1.0,2.0,0.0,3.0,4.0,1.0
2,Toronto C03,Humewood-Cedarvale,1038000.0,Semi-Detached,2-Storey,3.0,0.0,3,N,1.0,...,43.687827,-79.432772,2.0,53.0,6.0,2.0,0.0,4.0,0.0,1.0
3,Toronto C03,Oakwood-Vaughan,1145000.0,Store w/Apt/Offc,2-Storey,3.0,0.0,3,N,3.0,...,43.686992,-79.438444,2.0,86.0,7.0,2.0,0.0,4.0,3.0,1.0
4,Toronto C10,Mount Pleasant East,1149000.0,Detached,2-Storey,3.0,0.0,2,N,1.0,...,43.709162,-79.384864,8.0,78.0,1.0,2.0,0.0,4.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,Toronto W10,Thistletown-Beaumonde Heights,1049900.0,Detached,Bungalow,3.0,1.0,3,Y,1.0,...,43.741892,-79.578717,34.0,106.0,1.0,8.0,1.0,3.0,0.0,1.0
526,Toronto W10,Elms-Old Rexdale,1199999.0,Detached,1 1/2 Storey,3.0,2.0,2,Y,1.0,...,43.714159,-79.548880,34.0,38.0,1.0,0.0,1.0,3.0,0.0,1.0
527,Toronto W10,Mount Olive-Silverstone-Jamestown,1231000.0,Detached,Bungalow,4.0,2.0,2,Y,1.0,...,43.751754,-79.590279,34.0,77.0,1.0,8.0,1.0,3.0,0.0,1.0
528,Toronto W10,Mount Olive-Silverstone-Jamestown,1279000.0,Detached,2-Storey,4.0,1.0,3,N,1.0,...,43.746257,-79.591967,34.0,77.0,1.0,2.0,0.0,0.0,0.0,1.0


In [59]:
# see all columns
final2_df.columns

Index(['municipality_x', 'community_x', 'list_price', 'type_x', 'style_x',
       'br', 'er', 'wr', 'fr_x', 'kitchen', 'gar_type_x', 'ac_x', 'heat_x',
       'contract_date', 'lat', 'lng', 'municipality_y', 'community_y',
       'type_y', 'style_y', 'fr_y', 'gar_type_y', 'ac_y', 'heat_y'],
      dtype='object')

In [60]:
# drop unencoded columns
final2_df = final2_df.drop(['municipality_x', 'community_x', 'type_x', 'style_x', 'fr_x', 'gar_type_x', 'ac_x', 'heat_x'] ,1)


/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [61]:
final2_df

,list_price,br,er,wr,kitchen,contract_date,lat,lng,municipality_y,community_y,type_y,style_y,fr_y,gar_type_y,ac_y,heat_y
0,989000.0,4.0,2.0,2,2.0,1.639717e+09,43.773367,-79.448539,5.0,111.0,6.0,8.0,0.0,4.0,0.0,1.0
1,999900.0,3.0,0.0,2,1.0,1.641704e+09,43.682526,-79.442243,2.0,86.0,1.0,2.0,0.0,3.0,4.0,1.0
2,1038000.0,3.0,0.0,3,1.0,1.641791e+09,43.687827,-79.432772,2.0,53.0,6.0,2.0,0.0,4.0,0.0,1.0
3,1145000.0,3.0,0.0,3,3.0,1.634098e+09,43.686992,-79.438444,2.0,86.0,7.0,2.0,0.0,4.0,3.0,1.0
4,1149000.0,3.0,0.0,2,1.0,1.641877e+09,43.709162,-79.384864,8.0,78.0,1.0,2.0,0.0,4.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,1049900.0,3.0,1.0,3,1.0,1.641791e+09,43.741892,-79.578717,34.0,106.0,1.0,8.0,1.0,3.0,0.0,1.0
526,1199999.0,3.0,2.0,2,1.0,1.637989e+09,43.714159,-79.548880,34.0,38.0,1.0,0.0,1.0,3.0,0.0,1.0
527,1231000.0,4.0,2.0,2,1.0,1.641532e+09,43.751754,-79.590279,34.0,77.0,1.0,8.0,1.0,3.0,0.0,1.0
528,1279000.0,4.0,1.0,3,1.0,1.641618e+09,43.746257,-79.591967,34.0,77.0,1.0,2.0,0.0,0.0,0.0,1.0


In [62]:
# x values or features to predict on
X_active = final2_df.values
len(X_active)

530

In [63]:
# scale these values according to previous scaler
X_test_scaled2 = X_scaler.transform(X_active)

In [64]:
# use trained model to predict this data
y_predd2 = regressor.predict(X_test_scaled2)
df2 = pd.DataFrame({'Predicted': y_predd2})
df2

,Predicted
0,1.173075e+06
1,1.243484e+06
2,1.270424e+06
3,1.316758e+06
4,1.356093e+06
...,...
525,1.196823e+06
526,1.338049e+06
527,1.360546e+06
528,1.410150e+06


In [65]:
# add predicted values to predicted selling price column in active listings df
merge2_df['selling_price_predictions']=y_predd2
merge2_df

,municipality,community,list_price,type,style,br,er,wr,fr,kitchen,gar_type,ac,heat,contract_date,mls_id,address,lat,lng,selling_price_predictions
0,Toronto C07,Westminster-Branson,989000.0,Semi-Detached,Bungalow,4.0,2.0,2,N,2.0,None,Central Air,Gas,2021-12-17,C5457674,"8 Virgilwood Dr, Toronto",43.773367,-79.448539,1.173075e+06
1,Toronto C03,Oakwood-Vaughan,999900.0,Detached,2-Storey,3.0,0.0,2,N,1.0,Detached,Window Unit,Gas,2022-01-09,C5467990,"213 Northcliffe Blvd, Toronto",43.682526,-79.442243,1.243484e+06
2,Toronto C03,Humewood-Cedarvale,1038000.0,Semi-Detached,2-Storey,3.0,0.0,3,N,1.0,None,Central Air,Gas,2022-01-10,C5468205,"144 Winnett Ave, Toronto",43.687827,-79.432772,1.270424e+06
3,Toronto C03,Oakwood-Vaughan,1145000.0,Store w/Apt/Offc,2-Storey,3.0,0.0,3,N,3.0,None,Wall Unit,Gas,2021-10-13,C5400499,"357 Oakwood Ave, Toronto",43.686992,-79.438444,1.316758e+06
4,Toronto C10,Mount Pleasant East,1149000.0,Detached,2-Storey,3.0,0.0,2,N,1.0,None,Central Air,Gas,2022-01-11,C5469304,"82 Petman Ave, Toronto",43.709162,-79.384864,1.356093e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,Toronto W10,Thistletown-Beaumonde Heights,1049900.0,Detached,Bungalow,3.0,1.0,3,Y,1.0,Detached,Central Air,Gas,2022-01-10,W5468541,"9 Amaron Ave, Toronto",43.741892,-79.578717,1.196823e+06
526,Toronto W10,Elms-Old Rexdale,1199999.0,Detached,1 1/2 Storey,3.0,2.0,2,Y,1.0,Detached,Central Air,Gas,2021-11-27,W5443512,"35 Chilcot Ave, Toronto",43.714159,-79.548880,1.338049e+06
527,Toronto W10,Mount Olive-Silverstone-Jamestown,1231000.0,Detached,Bungalow,4.0,2.0,2,Y,1.0,Detached,Central Air,Gas,2022-01-07,W5467290,"145 Silverstone Dr, Toronto",43.751754,-79.590279,1.360546e+06
528,Toronto W10,Mount Olive-Silverstone-Jamestown,1279000.0,Detached,2-Storey,4.0,1.0,3,N,1.0,Attached,Central Air,Gas,2022-01-08,W5467611,"107 Mount Olive Dr, Toronto",43.746257,-79.591967,1.410150e+06


In [66]:
# change format of that column to non scientific format
merge2_df['selling_price_predictions']=merge2_df['selling_price_predictions'].apply(lambda x: '%.5f' % x)

In [67]:
merge2_df

,municipality,community,list_price,type,style,br,er,wr,fr,kitchen,gar_type,ac,heat,contract_date,mls_id,address,lat,lng,selling_price_predictions
0,Toronto C07,Westminster-Branson,989000.0,Semi-Detached,Bungalow,4.0,2.0,2,N,2.0,None,Central Air,Gas,2021-12-17,C5457674,"8 Virgilwood Dr, Toronto",43.773367,-79.448539,1173075.22406
1,Toronto C03,Oakwood-Vaughan,999900.0,Detached,2-Storey,3.0,0.0,2,N,1.0,Detached,Window Unit,Gas,2022-01-09,C5467990,"213 Northcliffe Blvd, Toronto",43.682526,-79.442243,1243483.93715
2,Toronto C03,Humewood-Cedarvale,1038000.0,Semi-Detached,2-Storey,3.0,0.0,3,N,1.0,None,Central Air,Gas,2022-01-10,C5468205,"144 Winnett Ave, Toronto",43.687827,-79.432772,1270424.40961
3,Toronto C03,Oakwood-Vaughan,1145000.0,Store w/Apt/Offc,2-Storey,3.0,0.0,3,N,3.0,None,Wall Unit,Gas,2021-10-13,C5400499,"357 Oakwood Ave, Toronto",43.686992,-79.438444,1316757.74736
4,Toronto C10,Mount Pleasant East,1149000.0,Detached,2-Storey,3.0,0.0,2,N,1.0,None,Central Air,Gas,2022-01-11,C5469304,"82 Petman Ave, Toronto",43.709162,-79.384864,1356092.99850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,Toronto W10,Thistletown-Beaumonde Heights,1049900.0,Detached,Bungalow,3.0,1.0,3,Y,1.0,Detached,Central Air,Gas,2022-01-10,W5468541,"9 Amaron Ave, Toronto",43.741892,-79.578717,1196822.64590
526,Toronto W10,Elms-Old Rexdale,1199999.0,Detached,1 1/2 Storey,3.0,2.0,2,Y,1.0,Detached,Central Air,Gas,2021-11-27,W5443512,"35 Chilcot Ave, Toronto",43.714159,-79.548880,1338049.19115
527,Toronto W10,Mount Olive-Silverstone-Jamestown,1231000.0,Detached,Bungalow,4.0,2.0,2,Y,1.0,Detached,Central Air,Gas,2022-01-07,W5467290,"145 Silverstone Dr, Toronto",43.751754,-79.590279,1360546.49839
528,Toronto W10,Mount Olive-Silverstone-Jamestown,1279000.0,Detached,2-Storey,4.0,1.0,3,N,1.0,Attached,Central Air,Gas,2022-01-08,W5467611,"107 Mount Olive Dr, Toronto",43.746257,-79.591967,1410150.05566


In [68]:
# output csv to resources cleaned column
output_data_file_active = "Resources_Cleaned/Active_Predictions.csv"
merge2_df.to_csv(output_data_file_active, index=False)